In [ ]:
import json
import os
from collections import defaultdict

In [ ]:
twohop_db = '/home/nirc/Arvind/neph_graph/greaselm_data/data_kg-umls2hop_q-sciel-noex/ddb'
threehop_db = '/home/nirc/Arvind/neph_graph/greaselm_data/data_kg-umls3hop_q-sciel-noex/ddb'
target_db = '/home/nirc/Arvind/neph_graph/greaselm_data/data_kg-umls23hop_q-sciel-noex/ddb'

In [ ]:
target_db_dir = f"{target_db}/ddb"
if not os.path.exists(target_db_dir):
    os.makedirs(target_db_dir)

In [ ]:
twohop_entities_file = os.path.join(twohop_db, "ddb_names.json")
twohop_relations_file = os.path.join(twohop_db, "ddb_relas.json")
twohop_ddb_to_umls_cui_file = os.path.join(twohop_db, "ddb_to_umls_cui.txt")

threehop_entities_file = os.path.join(threehop_db, "ddb_names.json")
threehop_relations_file = os.path.join(threehop_db, "ddb_relas.json")
threehop_ddb_to_umls_cui_file = os.path.join(threehop_db, "ddb_to_umls_cui.txt")

target_entities_file = os.path.join(target_db, "ddb_names.json")
target_relations_file = os.path.join(target_db, "ddb_relas.json")
target_ddb_to_umls_cui_file = os.path.join(target_db, "ddb_to_umls_cui.txt")

In [ ]:
with open(twohop_entities_file) as f:
    twohop_entities_dict = json.load(f)
with open(twohop_relations_file) as f:
    twohop_relations_dict = json.load(f)
with open(twohop_ddb_to_umls_cui_file) as f:
    twohop_ddb_to_umls_cui_list = f.read().splitlines()[1:]
    twohop_ddb_to_umls_cui_list = [l.split("\t") for l in twohop_ddb_to_umls_cui_list]
    
with open(threehop_entities_file) as f:
    threehop_entities_dict = json.load(f)
with open(threehop_relations_file) as f:
    threehop_relations_dict = json.load(f)
with open(threehop_ddb_to_umls_cui_file) as f:
    threehop_ddb_to_umls_cui_list = f.read().splitlines()[1:] 
    threehop_ddb_to_umls_cui_list = [l.split("\t") for l in threehop_ddb_to_umls_cui_list]

In [ ]:
db_entities_json = {}
db_to_umls = set()
umls_to_db =  dict()
db_entity_ids = defaultdict(lambda: len(db_entity_ids))

db_relations_json = {}
db_relation_ids = defaultdict(lambda: len(db_relation_ids))

In [ ]:
# add entity information
for _,str_db_id,cui,_ in twohop_ddb_to_umls_cui_list:
    db_id = db_entity_ids[cui]
    db_to_umls.add((db_id, cui))
    umls_to_db[cui] = db_id

    old_db_id = int(str_db_id)
    for name,(old_id,_) in twohop_entities_dict.items():
        if old_id == old_db_id:
            db_entities_json[name] = [db_id, "1"]

for _,str_db_id,cui,_ in threehop_ddb_to_umls_cui_list:
    db_id = db_entity_ids[cui]
    db_to_umls.add((db_id, cui))
    umls_to_db[cui] = db_id

    old_db_id = int(str_db_id)
    for name,(old_id,_) in threehop_entities_dict.items():
        if old_id == old_db_id:
            db_entities_json[name] = [db_id, "1"]

In [ ]:
threehop_entities_dict['Tacrolimus Bulk powder']

In [ ]:
# add relation information
for _,rel_tuple in twohop_relations_dict.items():
    (old_subj_id, old_obj_id, rel) = rel_tuple
    
    found_subj, found_obj = False, False
    for name,(old_id,_) in twohop_entities_dict.items():
        if found_subj and found_obj:
            break
        if old_id == old_subj_id:
            subj_id = db_entities_json[name][0]
            found_subj = True
        if old_id == old_obj_id:
            obj_id = db_entities_json[name][0]
            found_obj = True
    add_relation = (subj_id, obj_id, rel)
    relation_id = db_relation_ids[add_relation]
    db_relations_json[relation_id] = list(add_relation)
count = 0 
for _,rel_tuple in threehop_relations_dict.items():
    (old_subj_id, old_obj_id, rel) = rel_tuple
    
    found_subj, found_obj = False, False
    for name,(old_id,_) in threehop_entities_dict.items():
        if found_subj and found_obj:
            break
        if old_id == old_subj_id:
            subj_id = db_entities_json[name][0]
            found_subj = True
        if old_id == old_obj_id:
            obj_id = db_entities_json[name][0]
            found_obj = True
    add_relation = (subj_id, obj_id, rel)
    relation_id = db_relation_ids[add_relation]
    db_relations_json[relation_id] = list(add_relation)

In [ ]:
with open(target_ddb_to_umls_cui_file, 'w', encoding='utf-8') as f:
    f.write('\t'.join(["LinkItemsToUMLSCUIID", "ItemPTR", "CUI", "ItemToUMLSCUILinkTypePTR"]) + '\n')
    for row in sorted(list(db_to_umls)):
        db_ptr = row[0]
        cui = row[1]
        row = ["0", str(db_ptr), cui, "0"]
        f.write('\t'.join(row) + '\n')

with open(target_entities_file, 'w') as f:
    json.dump(db_entities_json, f)

with open(target_relations_file, 'w') as f:
    json.dump(db_relations_json, f)

In [ ]:
len(db_relations_json), len(twohop_relations_dict), len(threehop_relations_dict)